***Mount Google Drive at first!***

In [ ]:
# Uncomment line lower to copy project folder from gdrive to colab.(recommended)
!cp -av '/content/drive/MyDrive/TSBot_project' '/content'

In [ ]:
import os

# os.chdir('/content/drive/MyDrive/TSBot_project/')

# uncomment to set chdir to copied folder.(recommended)
os.chdir('/content/TSBot_project/')


In [ ]:
!pip install -r requirements.txt

***You need to restart colab after package installation.***

# Import packages and set constants

In [ ]:
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision.models as models
import logging
import shutil
from PIL import Image
import asyncio
from pathlib import Path
from aiogram import Bot, Dispatcher, types, executor
from aiogram.types import KeyboardButton, ReplyKeyboardMarkup, \
    InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardRemove
from aiogram.dispatcher.filters.state import StatesGroup, State
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from aiogram.dispatcher import FSMContext
from dotenv import load_dotenv
import warnings
import torchvision.transforms as transforms

warnings.simplefilter('ignore', UserWarning)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

# Define processing functions

In [ ]:
# Convert style names to CYCLEGAN's models name
style_dict = {'s2w': 'summer2winter_yosemite_pretrained', 'p2m': 'style_monet_pretrained',
              'p2c': 'style_cezanne_pretrained', 'p2u': 'style_ukiyoe_pretrained',
              'p2v': 'style_vangogh_pretrained'}


def style_processing_cyclegan(message_id):
    """The code of CYCLEGAN was written by [Jun-Yan Zhu](https://github.com/junyanz) and [Taesung Park](https://github.com/taesungp)
    , and supported by [Tongzhou Wang](https://github.com/SsnL).

    Run CYCLEGAN script with downloaded images from bot and return path to processed image"""
    style = message_id[-3:]
    style = style_dict[style]

    subprocess.call(f"cd model; python3 test.py --dataroot {os.getcwd()}/photos --results_dir "
                    f"{os.getcwd()}/photos --name {style} --model test --no_dropout", shell=True)

    return f'photos/{style}/test_latest/images/{message_id}_fake.png'


def sr_processing(message_id):
    """ Original model by: https://github.com/xinntao/ESRGAN
    Run SRGAN script with downloaded images from bot and return path to processed image"""
    id = message_id.split('_')[0]
    subprocess.call('cd ESRGAN; python3 test.py', shell=True)

    return f'ESRGAN/results/{message_id}_rlt.png'


def neural_transfer(user_id):
    '''Run neural style algorithm provided by PyTorch
    https://pytorch.org/tutorials/advanced/neural_style_tutorial.html'''
    loader = transforms.Compose([
        transforms.Resize(imsize),
        transforms.CenterCrop(imsize),  # scale imported image
        transforms.ToTensor()])  # transform it into a torch tensor

    def image_loader(image_name):
        image = Image.open(image_name)
        # fake batch dimension required to fit network's input dimensions
        image = loader(image).unsqueeze(0)
        return image.to(device, torch.float)

    style_path = f'Neural_transfer/{user_id}_style.jpg'
    cont_path = f'Neural_transfer/{user_id}_content.jpg'

    style_img = image_loader(style_path)
    content_img = image_loader(cont_path)

    assert style_img.size() == content_img.size(), \
        "we need to import style and content images of the same size"

    class ContentLoss(nn.Module):

        def __init__(self, target, ):
            super(ContentLoss, self).__init__()
            # we 'detach' the target content from the tree used
            # to dynamically compute the gradient: this is a stated value,
            # not a variable. Otherwise the forward method of the criterion
            # will throw an error.
            self.target = target.detach()

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

    def gram_matrix(input):
        a, b, c, d = input.size()  # a=batch size(=1)
        # b=number of feature maps
        # (c,d)=dimensions of a f. map (N=c*d)

        features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(a * b * c * d)

    class StyleLoss(nn.Module):

        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = gram_matrix(target_feature).detach()

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input

    cnn = models.vgg19(pretrained=True).features.to(device).eval()

    cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
    cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

    # create a module to normalize input image so we can easily put it in a
    # nn.Sequential
    class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view the mean and std to make them [C x 1 x 1] so that they can
            # directly work with image Tensor of shape [B x C x H x W].
            # B is batch size. C is number of channels. H is height and W is width.
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            # normalize img
            return (img - self.mean) / self.std


    def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                                   style_img, content_img,
                                   content_layers=None,
                                   style_layers=None):
        # normalization module
        if style_layers is None:
            style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
        if content_layers is None:
            content_layers = ['conv_4']
        normalization = Normalization(normalization_mean, normalization_std).to(device)

        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)

        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                name = 'relu_{}'.format(i)
                # The in-place version doesn't play very nicely with the ContentLoss
                # and StyleLoss we insert below. So we replace with out-of-place
                # ones here.
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                name = 'bn_{}'.format(i)
            else:
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module("style_loss_{}".format(i), style_loss)
                style_losses.append(style_loss)

        # now we trim off the layers after the last content and style losses
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses, content_losses

    def get_input_optimizer(input_img):
        # this line to show that input is a parameter that requires a gradient
        optimizer = optim.LBFGS([input_img])
        return optimizer

    def run_style_transfer(cnn, normalization_mean, normalization_std,
                           content_img, style_img, input_img, num_steps=300,
                           style_weight=800000, content_weight=40):
        """Run the style transfer."""
        print('Building the style transfer model..')
        model, style_losses, content_losses = get_style_model_and_losses(cnn,
                                                                         normalization_mean, normalization_std,
                                                                         style_img, content_img)

        # We want to optimize the input and not the model parameters so we
        # update all the requires_grad fields accordingly
        input_img.requires_grad_(True)
        model.requires_grad_(False)

        optimizer = get_input_optimizer(input_img)

        print('Optimizing..')
        run = [0]
        while run[0] <= num_steps:

            def closure():
                # correct the values of updated input image
                with torch.no_grad():
                    input_img.clamp_(0, 1)

                optimizer.zero_grad()
                model(input_img)
                style_score = 0
                content_score = 0

                for sl in style_losses:
                    style_score += sl.loss
                for cl in content_losses:
                    content_score += cl.loss

                style_score *= style_weight
                content_score *= content_weight

                loss = style_score + content_score
                loss.backward()

                run[0] += 1
                # if run[0] % 50 == 0:
                #     print("run {}:".format(run))
                #     print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                #         style_score.item(), content_score.item()))
                #     print()

                return style_score + content_score

            optimizer.step(closure)

        # a last correction...
        with torch.no_grad():
            input_img.clamp_(0, 1)

        return input_img

    input_img = content_img.clone()
    output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                content_img, style_img, input_img)
    # Save processed image
    out_arr = output.to('cpu').detach().numpy().squeeze(0)

    im = Image.fromarray((out_arr.transpose(1, 2, 0) * 255).astype(np.uint8))

    im.save(f"Neural_transfer/output_{cont_path.split('/')[-1][:-4]}.jpeg")

    return f"Neural_transfer/output_{cont_path.split('/')[-1][:-4]}.jpeg"


# Bot's main logic

In [ ]:
TOKEN = ''#Insert bot's token directly 
#or with venv/token.env file(only when project folder is on your local drive)

#load_dotenv('venv/token.env')
#TOKEN = os.getenv('TOKEN')

MAX_WIDTH_SR_INPUT = 1920
MAX_HEIGHT_SR_INPUT = 1080

# logging config
logging.basicConfig(filename='logs',
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logging.info("bot logging")

logger = logging.getLogger()

# Initialize bot
bot = Bot(token=TOKEN, parse_mode='HTML')
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

# Set dict of style names of bot
style_names = {'s2w': 'Summer⇄winter(Mountain view)', 'p2m': 'Photo⇄Monet(стиль Клода Моне)',
               'p2c': 'Photo⇄Cezanne(стиль Поля Сезанна)', 'p2u': 'Photo⇄Ukiyoe(стиль изобр. искусства Японии)',
               'p2v': 'Photo⇄Vangogh(стиль Винсента Ван Гога)', 'sr': 'Повышение разрешения изображения',
               'nt': 'Собственное изображение для переноса стиля'}

# Set buttons, Keyboard markup
b1 = KeyboardButton('/Выбрать_категорию')
b2 = KeyboardButton('/Продолжить_в_этом_стиле')
kb_client = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
kb_client.add(b1)

kb = InlineKeyboardMarkup(row_width=1, one_time_keyboard=True)
kb1 = InlineKeyboardButton(text=style_names['s2w'], callback_data='s2w')
kb2 = InlineKeyboardButton(text=style_names['p2m'], callback_data='p2m')
kb3 = InlineKeyboardButton(text=style_names['p2c'], callback_data='p2c')
kb4 = InlineKeyboardButton(text=style_names['p2u'], callback_data='p2u')
kb5 = InlineKeyboardButton(text=style_names['p2v'], callback_data='p2v')
kb6 = InlineKeyboardButton(text='Собственное изображение для переноса стиля', callback_data='nt')
kb7 = InlineKeyboardButton(text='Повышение разрешения изображения', callback_data='sr')

kb.add(kb1, kb2, kb3, kb4, kb5, kb6, kb7)


# Initialize bot states
class BotStates(StatesGroup):
    class NTState(StatesGroup):
        style = State()
        content = State()

    wait = State()
    SRGANState = State()
    CYCLEGANState = State()


'''' Start handlers '''


# Handler to not upload image while current is processing
@dp.message_handler(commands=['start', 'help', 'Выбрать_категорию'], state=BotStates.wait)
async def on_message(message: types.Message):
    await bot.send_message(message.from_user.id, '<strong>Одновременно можено обрабатывать '
                                                 'только одно изображение.</strong>')


# Start/help handler
@dp.message_handler(commands=['start', 'help'], state='*')
async def on_message(message: types.Message):
    await bot.send_message(message.from_user.id,
                           f'Привет, {message.from_user.first_name}! Я-TSBot, умею трансформировать стиль картинки в '
                           f'один из возможных стилей и повышать разрешение изображения.🖼️\n\n '
                           'Попробуйте сами, для этого нажмите кнопку "<b>Выбрать_категорию</b>".',
                           reply_markup=kb_client)


# Choose one category of bot's functional
@dp.message_handler(commands=['Выбрать_категорию'], state='*')
async def handle_docs_photo(message, state: FSMContext):
    await state.finish()
    await bot.send_message(message.from_user.id,
                           '<i><b>Выберите одну из категорий.🔠</b></i>',
                           reply_markup=kb)


# Catch callback from previous handler
@dp.callback_query_handler()
async def style_choose_handler(query: types.CallbackQuery):
    global cur_style_set
    cur_style_set = query.data
    await bot.delete_message(chat_id=query.from_user.id, message_id=query.message.message_id)
    await query.message.answer(f'Выбрана категория <i><b>"{style_names[cur_style_set]}".</b></i>\nТеперь загрузите '
                               f'картинку/фото, стиль которой(-го) хотите изменить.\n\n '
                               '<i><b>⚠Будьте осторожны!⚠</b></i>\n Бот принимает на вход квадратное изображение(для '
                               f'переноса стиля) и максимальное разрешение {MAX_WIDTH_SR_INPUT}*{MAX_HEIGHT_SR_INPUT} '
                               f'на входе (для режима повышения разрешения изображения)')

    # Choose state of one bot's styles
    if cur_style_set == 'nt':
        await query.message.answer('1️⃣Шаг 1. Загрузите изображениe, <i><b>стиль которого будет переноситься.</b></i>')
        await BotStates.NTState.style.set()

    elif cur_style_set == 'sr':
        await BotStates.SRGANState.set()

    else:
        await BotStates.CYCLEGANState.set()


'''SRGAN processing handler'''


# Download and process image with SRGAN
@dp.message_handler(state=BotStates.SRGANState, content_types=['photo'])
async def download_process_SRGAN(message):
    photo_name = f'{message.from_user.id}_{cur_style_set}'

    # Check if image is not bigger than MAX_HEIGHT_SR_INPUT * MAX_WIDTH_SR_INPUT else send error message
    if message.photo[-1]['width'] * message.photo[-1]['height'] <= MAX_HEIGHT_SR_INPUT * MAX_WIDTH_SR_INPUT:

        # Download photo
        await message.photo[-1].download(
            destination_file=Path(f'ESRGAN/LR/{photo_name}.jpg'))
        await bot.send_message(message.from_user.id,
                               '⌛Пожалуйста, подождите. Изображение обрабатывается...⌛',
                               reply_markup=ReplyKeyboardRemove())

        # Set wait state to not upload other images while current is processing
        await BotStates.wait.set()

        # Process the image
        output_path = await asyncio.get_running_loop().run_in_executor(None, sr_processing, photo_name)

        # Go to main state of this handler
        await BotStates.SRGANState.set()

        print('SRGAN:Done...')
        # Send processed image
        await bot.send_photo(chat_id=message.from_user.id, photo=open(output_path, 'rb'),
                             caption=f'✅Pезультат повышения разрешения изображения', )
        await bot.send_message(chat_id=message.from_user.id,
                               text=f'Чтобы продолжить <b><i>в этом стиле</i></b>, загрузите следующее изображение.\
                                \n\nЧтобы <b><i>сменить стиль</i></b>, нажмите кнопку '
                                    f'<b><i>"/Выбрать_категорию".</i></b>',
                               reply_markup=kb_client)

        # Delete downloaded and processed images
        try:
            os.remove(f'ESRGAN/LR/{photo_name}.jpg')
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))

        try:
            os.remove(f'ESRGAN/results/{photo_name}_rlt.png')
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))

    else:
        await bot.send_message(message.from_user.id,
                               f"<strong>⚠Загрузите фото с разрешeнием не выше "
                               f"{MAX_WIDTH_SR_INPUT}*{MAX_HEIGHT_SR_INPUT}⚠</strong>")


'''CycleGAN processing handler'''


# Download and process image with CycleGAN
@dp.message_handler(state=BotStates.CYCLEGANState, content_types=['photo'])
async def download_process_CYCLEGAN(message):
    photo_name = f'{message.from_user.id}_{cur_style_set}'

    # Get name of chosen style
    kb_name = style_names[cur_style_set]

    # Download photo
    await message.photo[-1].download(
        destination_file=Path(f'photos/{photo_name}.jpg'))
    await bot.send_message(message.from_user.id,
                           '⌛Пожалуйста, подождите. Изображение обрабатывается...⌛', reply_markup=ReplyKeyboardRemove())

    # Set wait state to not upload other images while current is processing
    await BotStates.wait.set()

    # Process the image
    output_path = await asyncio.get_running_loop().run_in_executor(None, style_processing_cyclegan, photo_name)

    # Go to main state of this handler
    await BotStates.CYCLEGANState.set()

    print('CycleGAN:Done...')
    # Send photo
    await bot.send_photo(chat_id=message.from_user.id, photo=open(output_path, 'rb'),
                         caption=f'✅Pезультат переноса стиля {kb_name}')
    await bot.send_message(chat_id=message.from_user.id,
                           text=f'Чтобы продолжить <b><i>в этом стиле</i></b>, загрузите следующее изображение.\
                                \n\nЧтобы <b><i>сменить стиль</i></b>, нажмите кнопку '
                                f'<b><i>"/Выбрать_категорию".</i></b>',
                           reply_markup=kb_client)

    # Delete downloaded and processed images
    try:
        os.remove(f'photos/{photo_name}.jpg')
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))

    try:
        shutil.rmtree(f'photos/{style_dict[cur_style_set]}')
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))


''' Neural Transfer process handlers '''


# Download and process image with slow algorithm
@dp.message_handler(state=BotStates.NTState.style, content_types='photo')
async def download_style_photo(message):
    # Download style image
    await message.photo[-1].download(
        destination_file=Path(f'Neural_transfer/{message.from_user.id}_style.jpg'))
    await bot.send_message(message.from_user.id,
                           f'2️⃣Шаг 2.Загрузите изображениe <i><b>на которое будет переноситься стиль.</b></i>')

    # Set state for download and process content image
    await BotStates.NTState.next()


@dp.message_handler(state=BotStates.NTState.content, content_types='photo')
async def download_content_process_photo(message):
    # Download content image
    await message.photo[-1].download(
        destination_file=Path(f'Neural_transfer/{message.from_user.id}_content.jpg'))

    await bot.send_message(message.from_user.id,
                           '⌛Пожалуйста, подождите. Изображение обрабатывается...⌛', reply_markup=ReplyKeyboardRemove())

    # Set wait state to not upload other images while current is processing
    await BotStates.wait.set()

    # Process the images
    output_path = await asyncio.get_running_loop().run_in_executor(None, neural_transfer, message.from_user.id)

    # Go to main state of this handler
    await BotStates.NTState.content.set()

    print('NT:Done...')
    await bot.send_photo(chat_id=message.from_user.id, photo=open(output_path, 'rb'),
                         caption=f'✅Pезультат переноса собственного стиля', )
    await bot.send_message(chat_id=message.from_user.id,
                           text=f'Чтобы продолжить <b><i>в этом стиле</i></b>, загрузите следующее изображение.\
                                \n\nЧтобы <b><i>сменить стиль</i></b>, '
                                f'нажмите кнопку <b><i>"/Выбрать_категорию".</i></b>',
                           reply_markup=kb_client)

    # Delete downloaded and processed images
    for path_to_remove in [f'Neural_transfer/{message.from_user.id}_style.jpg',
                           f'Neural_transfer/{message.from_user.id}_content.jpg', output_path]:
        try:
            os.remove(path_to_remove)
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))


''' Wrong file/command exception handlers '''


# Handler if user send photo as document. This need to avoid cases
# where file is too large and local server can drop while processing.
@dp.message_handler(content_types=['document'], state='*')
async def doc_handler(message: types.Message):
    await bot.send_message(message.from_user.id,
                           '😕Если это изображение, то отправьте его обычным способом.</strong>')


# Handler if user sent invalid commands or random words
@dp.message_handler(state='*')
async def cmd_not_found_handler(message: types.Message):
    await bot.send_message(message.from_user.id,
                           '😕Эта команда не поддерживаeтся на данный момент.')


# Handler if bot is just started and user wants to upload photo in no category.
@dp.message_handler(content_types=['photo'], state='*')
async def cat_not_found_handler(message: types.Message):
    await bot.send_message(message.from_user.id,
                           'Сначала выберите категорию.\nHажмите кнопку <i><b>"/Выбрать_категорию".</b></i>')

# Start polling

In [ ]:
import nest_asyncio

nest_asyncio.apply()
print('Bot is working...')
executor.start_polling(dp, skip_updates=True)